In [1]:
import pandas as pd
import numpy as np 
import requests
import json
from pprint import pprint
from config import key_dict
from datetime import datetime
import time
import gmaps

In [2]:
base_url = "http://api.openweathermap.org/data/2.5/air_pollution/history?"
key = key_dict['owm']
sd_lat = 32.7157
sd_lng = -117.1611

sf_lat = 37.7749
sf_lng = -122.4194

start = 1614585600 #march 1     #1521356400 #sunday march 18th 2018
end = 1616050800 #sunday march 18th 2021

In [3]:
sd_url = f"{base_url}lat={sd_lat}&lon={sd_lng}&start={start}&end={end}&appid={key}"

sd_data = requests.get(sd_url).json()

sf_url = f"{base_url}lat={sf_lat}&lon={sf_lng}&start={start}&end={end}&appid={key}"

sf_data = requests.get(sf_url).json()

In [4]:
sd_aqi_df = pd.DataFrame(columns = ['Date','Lat','Long','Components'])

for i in range(len(sd_data['list'])):

    components = sd_data['list'][i]['components']

    sd_aqi_df = sd_aqi_df.append({'Date':datetime.fromtimestamp(sd_data['list'][i]['dt']),
                                'Lat':sd_data['coord']['lat'], 'Long':sd_data['coord']['lon'],
                              'AQI':sd_data['list'][i]['main']['aqi'],
                                  'Components':sd_data['list'][i]['components']}, ignore_index = True)

sd_aqi_df.set_index('Date', inplace = True)    




In [5]:
sf_aqi_df = pd.DataFrame(columns = ['Date','Lat','Long','Components'])

for i in range(len(sf_data['list'])):

    components = sf_data['list'][i]['components']

    sf_aqi_df = sf_aqi_df.append({'Date':datetime.fromtimestamp(sf_data['list'][i]['dt']),
                                'Lat':sf_data['coord']['lat'], 'Long':sf_data['coord']['lon'],
                              'AQI':sf_data['list'][i]['main']['aqi'],
                                  'Components':sf_data['list'][i]['components']}, ignore_index = True)
    
sf_aqi_df.set_index('Date', inplace = True)
sf_aqi_df




,Lat,Long,Components,AQI
Date,,,,
2021-03-01 00:00:00,37.7749,-122.4194,"{'co': 961.3, 'no': 61.69, 'no2': 85, 'o3': 0,...",4.0
2021-03-01 01:00:00,37.7749,-122.4194,"{'co': 988.01, 'no': 64.37, 'no2': 86.37, 'o3'...",4.0
2021-03-01 02:00:00,37.7749,-122.4194,"{'co': 921.25, 'no': 56.77, 'no2': 76.08, 'o3'...",4.0
2021-03-01 03:00:00,37.7749,-122.4194,"{'co': 881.2, 'no': 49.62, 'no2': 71.29, 'o3':...",4.0
2021-03-01 04:00:00,37.7749,-122.4194,"{'co': 867.84, 'no': 40.68, 'no2': 74.71, 'o3'...",4.0
...,...,...,...,...
2021-03-17 20:00:00,37.7749,-122.4194,"{'co': 253.68, 'no': 0, 'no2': 9.25, 'o3': 92....",2.0
2021-03-17 21:00:00,37.7749,-122.4194,"{'co': 260.35, 'no': 0, 'no2': 10.88, 'o3': 90...",2.0
2021-03-17 22:00:00,37.7749,-122.4194,"{'co': 263.69, 'no': 0, 'no2': 11.82, 'o3': 88...",2.0


In [12]:
test_final_df = sf_aqi_df.append(sd_aqi_df).sort_index()
test_final_df = test_final_df.loc[test_final_df.index.hour == 0,:]

In [52]:
test_final_df['AQI']

Date
2021-03-01    4.0
2021-03-01    2.0
2021-03-02    4.0
2021-03-02    4.0
2021-03-03    4.0
2021-03-03    5.0
2021-03-04    1.0
2021-03-04    2.0
2021-03-05    2.0
2021-03-05    4.0
2021-03-06    2.0
2021-03-06    4.0
2021-03-07    1.0
2021-03-07    1.0
2021-03-08    1.0
2021-03-08    2.0
2021-03-09    2.0
2021-03-09    1.0
2021-03-10    2.0
2021-03-10    2.0
2021-03-11    2.0
2021-03-11    2.0
2021-03-12    1.0
2021-03-12    4.0
2021-03-13    4.0
2021-03-13    2.0
2021-03-14    2.0
2021-03-14    1.0
2021-03-15    2.0
2021-03-15    1.0
2021-03-16    1.0
2021-03-16    2.0
2021-03-17    3.0
2021-03-17    2.0
2021-03-18    2.0
2021-03-18    2.0
Name: AQI, dtype: float64

In [9]:
from ipywidgets.embed import embed_minimal_html
from IPython.display import display

gmaps.configure(api_key = key_dict['google'])

In [55]:

# credit to pbugnion on github; small adjustments for personal keys/indexes, accepting dataframe format, etc
# https://github.com/pbugnion/gmaps/issues/216


class HeatmapAnimation(object):
    
    def __init__(self, datasets):
        self._datasets = datasets
        self._figure = gmaps.figure(center=(37.0, -120.0), zoom_level=5)
        self._index_count = 0
        self._current_index = datasets.index.unique()[self._index_count]
        self._heatmap = gmaps.heatmap_layer(locations = datasets.loc[datasets.index == self._current_index,['Lat','Long']],
                                            weights = datasets.loc[datasets.index == self._current_index,'AQI'],
                                           max_intensity = datasets['AQI'].max(), point_radius = 30)
        self._figure.add_layer(self._heatmap)
        
    def render(self):
        return display(self._figure)
    
    def start_animation(self):
        while True:
            self._index_count = self._index_count + 1
            try:
                self._current_index = self._datasets.index.unique()[self._index_count]
            except IndexError:
                self._index_count = 0
                self._current_index = self._datasets.index.unique()[self._index_count]
            self._render_current_dataset()
            time.sleep(0.25)
    
    def _render_current_dataset(self):
        self._heatmap.locations = self._datasets.loc[self._datasets.index == 
                                               self._current_index,['Lat','Long']] # update the locations drawn on the heatmap
        self._heatmap.weights = self._datasets.loc[self._datasets.index == 
                                             self._current_index,'AQI']

        



In [57]:
animation = HeatmapAnimation(test_final_df)
animation.render()
animation.start_animation()


Figure(layout=FigureLayout(height='420px'))

KeyboardInterrupt: 

In [40]:
#now figure out timesieries heatmap
# don't run this until you can figure out WHAT calls the api key. If it's just the base map, you're fine.
# if it's the heatmap layer too, that's a problem. 



#I've turned this cell into text for safety purposes so I don't accidentally run up a huge number of api calls

"""
#update 1am 3/23/21, I believe the next form of action if I want to generate a heatmap that changes over time
is to plot heatmap style colors individually, rather than rely on the heatmap function. data would look like
(x, y, intensity) sbut only take in 1 set of values from the df, where we've already computed the relative intensity 
on a 0-1 scale or something like that. We can then create a range of colors, from green to red (there's probably a color 
pack for this), and plotthose colors individually through a loop 
"""

fig = gmaps.figure()


location = test_final_df[['Lat_sf','Long_sf'][0],['Lat_sd','Long_sd'][0]]


aqi_layer = gmaps.heatmap_layer(location, weights = aqi_df.iloc[0,2],
                                max_intensity = aqi_df['AQI'].max(),
                               dissipating = False)
                               
                               #I believe the above doesn't work because it's looking for multiple locations and weights
                               
                               
fig.add_layer(aqi_layer)

fig
time.sleep(0.2)


KeyError: ('Lat_sf', 'Lat_sd')

In [43]:
test_figure = gmaps.figure()

location = test_final_df.loc[test_final_df.index == pd.Timestamp('2021-03-01 00:00:00'),['Lat','Long']]

weights = test_final_df.loc[test_final_df.index == pd.Timestamp('2021-03-01 00:00:00'), 'AQI']

max_intensity = test_final_df.loc[:, 'AQI'].max()


heatlayer = gmaps.heatmap_layer(locations = location, weights = weights, max_intensity = max_intensity,
                               dissipating = False, point_radius = 1)

test_figure.add_layer(heatlayer)
test_figure

Figure(layout=FigureLayout(height='420px'))

5.0

In [44]:
heatlayer.weights = [2.0,2]